# Colour viewer for HOYS lightcurves

* ***Shift + Enter on a code cell to run it and advance to the next cell.***

* ***If changing sliders, rerun the code block below the sliders***

* ***Plot windows are interactive, can use mouse to select and zoom into ranges***

In [ ]:
#load required modules
import os
import pandas as pd
import astropy
import numpy
import math
from astropy.time import Time
from astropy.timeseries import LombScargle
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import iplot
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display,clear_output

### List available lightcurve data and select from dropdown list
* Upload your .csv file to '/light_curve_csv_files'

In [ ]:
#read HOYS csv file
lc_folder = "./light_curve_csv_files/"
star_list=os.listdir(lc_folder)

lc_select = widgets.Dropdown(
    options=sorted(star_list),
    description='Select a light curve file:',
    style = {'description_width': 'initial'}
)
display(lc_select)

### Display head and tail of csv file containing all of the data

In [ ]:
print('selected datafile:',lc_select.value)
lc_data = pd.read_csv(os.path.join(lc_folder,lc_select.value),comment='#',delimiter=' ')
pd.set_option('display.max_columns', None)
#view file head and tail
lc_data

In [ ]:
filter_list=numpy.unique(lc_data['filter'])

# filter_select = widgets.SelectMultiple(
#     options=sorted(filter_list),
#     value=['B','V','R','I'],
#     description='Select a filter',
#     style = {'description_width': 'initial'}
# )
# display(filter_select)

jd=Time(lc_data.date,format='jd') 
mjd=jd.mjd #plot mjd instead of jd
date_range_sel=widgets.FloatRangeSlider(
    value=[min(mjd),max(mjd)],
    min=min(mjd),
    max=max(mjd),
    step=0.1,
    description='Date range [mjd]:',
    readout_format='.1f',
    layout={'width': '600px'},
    style = {'description_width': 'initial'}

)
display(date_range_sel)

cal_error_sel=widgets.FloatSlider(
    value=0.25,
    min=0.1,
    max=0.4,
    step=0.01,
    description='Calibrated error [mag]:',
    layout={'width': '600px'},
    style = {'description_width': 'initial'}
)
display(cal_error_sel)

med_window_sel=widgets.IntSlider(
    value=1,
    min=1,
    max=10,
    step=1,
    description='Median filter window [days]:',
    layout={'width': '400px'},
    style = {'description_width': 'initial'}
)
display(med_window_sel)

In [ ]:
#select data from specifed filter and remove bad data
#band=filter_select.value
band=['B','V','R','I']
cal_error=cal_error_sel.value
med_window=med_window_sel.value
date_range=Time(date_range_sel.value,format='mjd').jd

filtered_data=lc_data[(lc_data['filter'].isin(band)) & (lc_data['flags'] <= 4) & (lc_data['calibrated_error'] < cal_error) & (lc_data['date'] > date_range[0]) & (lc_data['date'] < date_range[1]) & (lc_data['calibrated_magnitude'] > 0.0) & (lc_data['calibrated_error'] > 0.0) & (lc_data['fwhm_world'] > 0.0) & (lc_data['fwhm_world'] < 9.0/3600.0)]
filtered_data.reset_index(inplace=True,drop=True)
#filtered_data['utc']=pd.to_datetime(filtered_data['date'],unit='D', origin='julian')

#start=Time.now()

# slow method in pandas
# for date in filtered_data.index:
#     med_mask=(abs(filtered_data['date'][date] - filtered_data['date']) < 1)
#     for fil in band:
#         filtered_data.loc[med_mask,fil]=numpy.median(filtered_data[med_mask]['calibrated_magnitude'][(filtered_data[med_mask]['filter']==fil)])

#fast method with numpy arrays
date=numpy.array(filtered_data['date'])
filt=numpy.array(filtered_data['filter'])
mag=numpy.array(filtered_data['calibrated_magnitude'])
dt2=numpy.dtype([('B',numpy.float32),('V',numpy.float32),('R',numpy.float32),('I',numpy.float32),
                ('Be',numpy.float32),('Ve',numpy.float32),('Re',numpy.float32),('Ie',numpy.float32)])
colours=numpy.empty(mag.shape,dtype=dt2)
colours[:]=numpy.nan
for i in range(0,len(filtered_data)):
    for f in band:
        check = numpy.where( (filt == f) &  (numpy.abs(date[i] - date) < med_window) )
        if len(check[0]>0):
            colours[i][f]=numpy.median(mag[check[0]])
            colours[i][f+'e'] = numpy.nanstd(mag[check[0]])/numpy.sqrt(float(len(mag[check[0]])))

        
filtered_data=pd.concat((filtered_data,pd.DataFrame(colours)),axis=1)

display(filtered_data.sort_values(by='date'))

#end=Time.now()
#print((end-start).sec)

### Lightcurve plot

In [ ]:
def plot_lightcurve(mag_range_select=[min(filtered_data['calibrated_magnitude']),max(filtered_data['calibrated_magnitude'])],error_bars=False):
    global mag_data
    mag_range=mag_range_select
    mag_data=filtered_data[(filtered_data['calibrated_magnitude'] > mag_range[0]) & (filtered_data['calibrated_magnitude'] < mag_range[1])]

    jd=Time(mag_data.date,format='jd') 
    mjd=jd.mjd #plot mjd instead of jd
    
    color_discrete_map = {'B': 'blue', 'V': 'green', 'R': 'red', 'I':'black'}

    #plot lightcurve
    if error_bars==True:
        fig1=px.scatter(mag_data,x=mjd,y=mag_data.calibrated_magnitude,color='filter',color_discrete_map=color_discrete_map,labels={'x':'Date [mjd]'},
                       error_y=mag_data.calibrated_error)
    else:
        fig1=px.scatter(mag_data,x=mjd,y=mag_data.calibrated_magnitude,color='filter',color_discrete_map=color_discrete_map,labels={'x':'Date [mjd]'})
    fig1['layout']['yaxis_title']='Magnitude [mag]'
    fig1['layout']['yaxis']['autorange'] = "reversed"
    fig1['layout']['xaxis']['tickformat'] = "3f"

    iplot(fig1)
    
interact(plot_lightcurve,mag_range_select=widgets.FloatRangeSlider(
    min=min(filtered_data['calibrated_magnitude']),max=max(filtered_data['calibrated_magnitude']),
    step=0.01,value=[min(filtered_data['calibrated_magnitude']),max(filtered_data['calibrated_magnitude'])],
    layout={'width': '600px'},description='Magnitude range [mag]',style = {'description_width': 'initial'},continuous_update=False
))

### Colour curve plot

In [ ]:
def view_colourcurve(col1='V',col2='I',error_bars=False):
    jd=Time(filtered_data.date,format='jd') 
    mjd=jd.mjd #plot mjd instead of jd

    if error_bars==True:
        fig1=px.scatter(filtered_data,x=mjd,y=filtered_data[col1]-filtered_data[col2],labels={'x':'Date [mjd]', 'y':f'{col1}-{col2} [mag]'},
                    error_y=numpy.sqrt(numpy.square(filtered_data[col1+'e']) + numpy.square(filtered_data[col2+'e'])))
    else:
        fig1=px.scatter(filtered_data,x=mjd,y=filtered_data[col1]-filtered_data[col2],labels={'x':'Date [mjd]', 'y':f'{col1}-{col2} [mag]'})
    fig1['layout']['xaxis']['tickformat'] = "3f"

    iplot(fig1)

interact(view_colourcurve,col1= widgets.Dropdown(options=sorted(band),value='V',description='Select first colour',style = {'description_width': 'initial'}),
                          col2= widgets.Dropdown(options=sorted(band),value='I',description='Select second colour',style = {'description_width': 'initial'}))


### Colour-magnitude plot

In [ ]:
def plot_cmd(m1='V',col1='V',col2='I',error_bars=False):
    jd=Time(filtered_data.date,format='jd') 
    mjd=jd.mjd #plot mjd instead of jd
    if error_bars==True:
        fig1=px.scatter(filtered_data,x=filtered_data[col1]-filtered_data[col2],y=filtered_data[m1],color=mjd,
                   error_y=numpy.sqrt(numpy.square(filtered_data[col1+'e']) + numpy.square(filtered_data[col2+'e'])),error_x=filtered_data[m1+'e'])
    else:
        fig1=px.scatter(filtered_data,x=filtered_data[col1]-filtered_data[col2],y=filtered_data[m1],color=mjd)

    x = numpy.linspace(-1, 0, 100)
    y = 3.1 * x +numpy.max(filtered_data[m1])
    fig2=px.line(x=x+numpy.max(filtered_data[col1]-filtered_data[col2]),y=y,labels='R_v=3.1')
    fig2.update_traces(line=dict(color = 'blue',width=3))
    
    x = numpy.linspace(-1, 0, 100)
    y = 5 * x +numpy.max(filtered_data[m1])
    fig3=px.line(x=x+numpy.max(filtered_data[col1]-filtered_data[col2]),y=y,labels='R_v=5')
    fig3.update_traces(line=dict(color = 'red',width=3))
    
    fig4 = go.Figure(data=fig2.data + fig3.data + fig1.data)
    fig4['layout']['yaxis_title']=f'Magnitude ({m1}) [mag]'
    fig4['layout']['xaxis_title']=f'Colour ({col1}-{col2}) [mag]'
    fig4['layout']['yaxis']['autorange'] = "reversed"
    fig4['layout']['coloraxis_colorbar']['title'] = 'Date [mjd]'
    fig4.update_layout(autosize=False,width=900,height=600)
    
    fig4.show()
    #iplot(fig1)
    
interact(plot_cmd,m1= widgets.Dropdown(options=sorted(band),value='V',description='Select magnitude',style = {'description_width': 'initial'}),
                         col1= widgets.Dropdown(options=sorted(band),value='V',description='Select first colour',style = {'description_width': 'initial'}),
                          col2= widgets.Dropdown(options=sorted(band),value='I',description='Select second colour',style = {'description_width': 'initial'}))
   